## Load libraries

In [1]:
import pandas as pd
df = pd.read_csv('FCG Germany GmbH_Data Scientist_case study_Inspections.csv')
df_auctions = pd.read_csv('FCG Germany GmbH_Data Scientist_case study_Auctions.csv')

In [2]:
import json
import numpy as np
from datetime import datetime

# json object to pandas 

'inspection_report', 'category_score'

add: inspection_report

In [3]:
def generrate_inspection(row, column_name, the_key):
    json_data = row[column_name]#['inspection_report']
    try:
        python_obj = json.loads(json_data)
        if the_key in python_obj.keys():
            return python_obj[the_key]
        else:
            return float("nan")
    except:
        return float("nan")

In [4]:
## This take long to run:
## Run to add features about inspection_report
column_name = 'inspection_report'
df_extend = df.copy()
insp_json_data = df.iloc[0][column_name]#.inspection_report
for thekey in list(json.loads(insp_json_data).keys()):
    col_name = 'insp_rep_' + thekey
    df_extend[col_name] = df_extend.apply( generrate_inspection, args=[column_name, thekey], axis = 1) 

add: category_scores as pandas

In [31]:
def generrate_cat_data(row, column_name, the_key):
    json_data = row[column_name]
    try:
        tmp_df = pd.DataFrame(json.loads(json_data)).set_index('id')
        if the_key in list(tmp_df.index):
            return tmp_df.loc[the_key]['score']
        else:
            return float("nan")
    except:
        return float("nan")

In [37]:
## This take long to run:
## Run to add features about category_scores
column_name2 = 'category_scores'
df_extend_test = df_extend.copy()
cate_json_data = df.iloc[0][column_name2]
for doc in list(json.loads(cate_json_data)) :
    col_name = 'cat_score_' + doc['id']
    df_extend_test[col_name] = df_extend_test.apply( generrate_cat_data, args=[column_name2, doc['id']], axis = 1)

## Drop some columns

In [46]:
df_extend_test.shape, df_extend_load.shape

((21160, 215), (21160, 219))

In [50]:
# columns to drop: with a lot null values, consider drop these column
col_alot_null = ['deleted_at', 'end_time', 'insp_rep_NIK', 'insp_rep_paintscannerFrontBumper']
df_extend_load = df_extend_load.drop(col_alot_null, axis=1)

In [67]:
df_extend_test = df_extend_load.copy()
df_extend_test.shape

(21160, 215)

In [68]:
# columns to fillna -1: car_id, insp_rep_priceInspectorExpectation

df_extend_test['car_id'] = df_extend_test['car_id'].fillna(-1)
df_extend_test['insp_rep_priceInspectorExpectation'] = df_extend_test['insp_rep_priceInspectorExpectation'].fillna(-1)


In [69]:
df_extend_test.shape

(21160, 215)

## TF to Boolean values
Convert True False to 1 0<br>
how to deal with NAN ?

In [79]:
list_bool_keys = []
for key, val in json.loads(df.iloc[0].inspection_report).items():
    if type(val) == type(True):
        #print (key, val, type(val))
        list_bool_keys += ['insp_rep_'+key]

In [87]:
len(list_bool_keys)
for iname in df_extend_test.columns:
    if iname in list_bool_keys: 
        df_extend_test[iname] = df_extend_test[iname].astype(int)

## Datetime column
- Convert datetime object

In [94]:
#Datetime
# inspection_date, insp_rep_tax
df_extend_test['inspection_date']= pd.to_datetime(df_extend_test['inspection_date']) 

In [95]:
def str_my_to_datetime(row):
    try:
        out = datetime.strptime(row['insp_rep_tax'], '%m/%Y')
        return out
    except: 
        return datetime(5000,12,31)

In [96]:
df_extend_test['insp_rep_tax'] = df_extend_test.apply( str_my_to_datetime , axis = 1)

Further checking column type

In [99]:
# on, off, n/a
list_dummie_onoffna = [
    'insp_rep_blower',
    'insp_rep_armRest',
    'insp_rep_heating',
    'insp_rep_headRest',
    'insp_rep_fogLightLeft',
    'insp_rep_discBreakLeft',
    'insp_rep_fogLightRight',
    'insp_rep_discBreakRight',
    'insp_rep_dashControlDeFog',
    'insp_rep_matchingVinNumber',
    'insp_rep_trunkDoorFunction',
    'insp_rep_extraMonitorFunction',
    'insp_rep_matchingEngineNumber',
    'insp_rep_sideMirrorLeftAutoRetracts',
    'insp_rep_sideMirrorRightAutoRetracts'
]

In [100]:
def replace_na_with_notavail(row, col_name):
    try:
        if row[col_name] == 'n/a':
            return 'notavail'
        else:
            return row[col_name]
    except: 
        return 'notavail'

In [101]:
for onoffvar in list_dummie_onoffna:
    df_extend_test[onoffvar] = df_extend_test.apply( replace_na_with_notavail , args = [onoffvar], axis = 1)

## Categorical Vars
to onehotencoding, drop useless variables

In [106]:
# one hot encoding lists
list_dummie = []

# categorical
list_dummie += [
    'inspection_leadsource',
    'insp_rep_bpkb',
    'insp_rep_make',
    'insp_rep_model',
    'insp_rep_ownership'
]

# on, off, n/a
list_dummie += list_dummie_onoffna

# other categorical
list_dummie += [
    'insp_rep_fuel',
    'insp_rep_leadsource',
    'insp_rep_formAOption',
    'insp_rep_chassisColor',
    'insp_rep_transmission',
    'insp_rep_frontAirbagOption',
    'insp_rep_1stOwnershipOption'
]

In [108]:
# will drop : insp_rep_trim : e.g. '1.5 RS Bensin' >> seems not useful
df_extend_test = df_extend_test.drop('insp_rep_trim', axis=1)

In [109]:
# one hot coding the other categorical vars
dfDummies = pd.get_dummies(df_extend_test[list_dummie])
df_extend_test = pd.concat([df_extend_test, dfDummies], axis=1)

## Join table with price

In [116]:
df_model_auctions = pd.merge(df_auctions, df_model, on='car_id', how='inner')

## Add column : how old is the car?
auction_start_date - manufactured_year<br>
year_to_date

In [121]:
df_model_auctions['auction_start_date']= pd.to_datetime(df_model_auctions['auction_start_date']) 
df_model_auctions['auction_start_date'].iloc[0]

Timestamp('2019-08-03 12:20:00')

In [122]:
def make_timediff_year (row, varpast, varnow):
    try:
        ans_years = row[varnow].year - row[varpast]
        return ans_years
    except: 
        return float('nan')


In [124]:
df_model_auctions['year_to_date'] = df_model_auctions.apply( 
        make_timediff_year , args = ['insp_rep_year','auction_start_date'], axis = 1
        )

## Drop unused columns

In [128]:
# drop unused column
# from inspection table
unused_cols = []
unused_cols += [
    'inspection_id',
    'car_id',
    'inspection_date',
    'category_scores',
    'inspection_report',
    'start_time',
    'insp_rep_tax',
    'insp_rep_year',
    'insp_rep_lastService',
    
]
unused_cols += list_dummie

# from auction table
unused_cols += [
    'auction_id',
    'auction_created_date',
    'auction_start_date',
    'auction_original_end_date',
    'auction_end_date',
    'buy_now_enabled',
    'winning_bid_price',
    'auction_buy_now_price',
    'won_with_buy_now',
    'winner_dealer_id',
    'has_winner',
    'number_of_bids',
    'number_of_bidders'
]


In [129]:
df_model_auctions = df_model_auctions.drop(unused_cols, axis=1)

In [133]:
# These columns should be 0 if it's empty ''
str_s = [
    'returned', #bool
    'insp_rep_keySets',
    'insp_rep_cylinderCapacity',
    'insp_rep_priceSellerExpectation',
    'insp_rep_priceInspectorExpectation',
]

In [134]:
for str_ in str_s: 
    df_model_auctions[str_] = df_model_auctions[str_].apply(lambda x: 0 if x =='' else x)
    df_model_auctions[str_] = df_model_auctions[str_].astype(int)

## Price columns

In [136]:
df_model_auctions = df_model_auctions[(df_model_auctions['insp_rep_priceSellerExpectation'] > 0) & 
                 (df_model_auctions['insp_rep_priceInspectorExpectation'] > 0)]
df_model_auctions = df_model_auctions.reset_index(drop = True)
# I think these price should be imposrtant so drop those 49 obs that has no info

In [172]:
df_model_auctions = df_model_auctions.dropna()

## Save pandas

In [173]:
df_model_auctions.to_csv('used_car_extend_model_clean.csv',index=False)